![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © Carlos A. Iglesias

# Semantic Models

# Table of Contents
* [Objectives](#Objectives)
* [Corpus](#Corpus)
* [Converting Scikit-learn to gensim](#Converting-Scikit-learn-to-gensim)
* [Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation-%28LDA%29)
* [Latent Semantic Indexing (LSI)](#Latent-Semantic-Indexing-%28LSI%29)

# Objectives

In this session we provide a quick overview of the semantic models presented during the classes. In this case, we will use a real corpus so that we can extract meaningful patterns.

The main objectives of this session are:
* Understand the models and their differences
* Learn to use some of the most popular NLP libraries

# Corpus

We are going to use on of the corpus that come prepackaged with Scikit-learn: the [20 newsgroup datase](http://qwone.com/~jason/20Newsgroups/). The 20  newsgroup dataset contains 20k documents that belong to 20 topics.

We inspect now the corpus using the facilities from Scikit-learn, as explain in [scikit-learn](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html#newsgroups)

In [1]:
from sklearn.datasets import fetch_20newsgroups

# We filter only some categories, otherwise we have 20 categories
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# We remove metadata to avoid bias in the classification
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'), 
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                                    categories=categories)


# Obtain a vector

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', min_df=10)

vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_train.shape

(2034, 2807)

# Converting Scikit-learn to gensim

Although scikit-learn provides an LDA implementation, it is more popular the package *gensim*, which also provides an LSI implementation, as well as other functionalities. Fortunately, scikit-learn sparse matrices can be used in Gensim using the function *matutils.Sparse2Corpus()*. Anyway, if you are using intensively LDA,it can be convenient to create the corpus with their functions.

You should install first *gensim*. Run 'conda install -c anaconda gensim=0.12.4' in a terminal.

In [2]:
from gensim import matutils

vocab = vectorizer.get_feature_names()

dictionary = dict([(i, s) for i, s in enumerate(vectorizer.get_feature_names())])
corpus_tfidf = matutils.Sparse2Corpus(vectors_train)

# Latent Dirichlet Allocation (LDA)

Although scikit-learn provides an LDA implementation, it is more popular the package *gensim*, which also provides an LSI implementation, as well as other functionalities. Fortunately, scikit-learn sparse matrices can be used in Gensim using the function *matutils.Sparse2Corpus()*.

In [3]:
from gensim.models.ldamodel import LdaModel

# It takes a long time

#  train the lda model, choosing number of topics equal to 4
lda = LdaModel(corpus_tfidf, num_topics=4, passes=20, id2word=dictionary)

In [4]:
# check the topics
lda.print_topics(4)

[(0,
  '0.004*"objects" + 0.003*"manhattan" + 0.003*"obtained" + 0.002*"eyes" + 0.002*"education" + 0.002*"activity" + 0.002*"men" + 0.002*"2000" + 0.002*"chosen" + 0.002*"dark"'),
 (1,
  '0.005*"activity" + 0.005*"complex" + 0.004*"conclusions" + 0.004*"professional" + 0.004*"netters" + 0.004*"defined" + 0.004*"agreed" + 0.004*"choose" + 0.003*"potential" + 0.003*"looking"'),
 (2,
  '0.006*"archie" + 0.006*"car" + 0.006*"increased" + 0.006*"closely" + 0.006*"beginning" + 0.006*"comets" + 0.006*"groups" + 0.006*"center" + 0.005*"dealing" + 0.005*"88"'),
 (3,
  '0.003*"led" + 0.003*"mechanism" + 0.003*"platform" + 0.003*"frank" + 0.002*"mormons" + 0.002*"aeronautics" + 0.002*"concepts" + 0.002*"header" + 0.002*"proton" + 0.002*"foreign"')]

Since there are some problems for translating the corpus from Scikit-Learn to LSI, we are now going to create 'natively' the corpus with Gensim.

In [5]:
# import the gensim.corpora module to generate dictionary
from gensim import corpora

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import RegexpTokenizer

import string

def preprocess(words):
    tokenizer = RegexpTokenizer('[A-Z]\w+')
    tokens = [w.lower() for w in tokenizer.tokenize(words)]
    stoplist = stopwords.words('english')
    tokens_stop = [w for w in tokens if w not in stoplist]
    punctuation = set(string.punctuation)
    tokens_clean = [w for w in tokens_stop if  w not in punctuation]
    return tokens_clean

#words = preprocess(newsgroups_train.data)
#dictionary = corpora.Dictionary(newsgroups_train.data)

texts = [preprocess(document) for document in newsgroups_train.data]

dictionary = corpora.Dictionary(texts)

In [6]:
# You can save the dictionary
dictionary.save('newsgroup.dict')

print(dictionary)

Dictionary(10913 unique tokens: ['cel', 'ds', 'hi', 'nothing', 'prj']...)


In [7]:
# Generate a list of docs, where each doc is a list of words

docs = [preprocess(doc) for doc in newsgroups_train.data]

In [8]:
# import the gensim.corpora module to generate dictionary
from gensim import corpora

dictionary = corpora.Dictionary(docs)

In [9]:
# You can optionally save the  dictionary 

dictionary.save('newsgroups.dict')
lda = LdaModel.load('newsgroups.dict')

In [10]:
# We can print the dictionary, it is a mappying of id and tokens

print(dictionary)

Dictionary(10913 unique tokens: ['cel', 'ds', 'hi', 'nothing', 'prj']...)


In [11]:
# construct the corpus representing each document as a bag-of-words (bow) vector
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
from gensim.models import TfidfModel

# calculate tfidf
tfidf_model = TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

In [13]:
#print tf-idf of first document
print(corpus_tfidf[0])

[(0, 0.24093628445650234), (1, 0.5700978153855775), (2, 0.10438175896914427), (3, 0.1598114653031772), (4, 0.722808853369507), (5, 0.24093628445650234)]


In [14]:
from gensim.models.ldamodel import LdaModel

# train the lda model, choosing number of topics equal to 4, it takes a long time

lda_model = LdaModel(corpus_tfidf, num_topics=4, passes=20, id2word=dictionary)

In [15]:
# check the topics
lda_model.print_topics(4)

[(0,
  '0.008*"western" + 0.008*"plane" + 0.007*"unix" + 0.006*"davidian" + 0.006*"bob" + 0.006*"unfortunately" + 0.005*"sorry" + 0.005*"hi" + 0.005*"rayshade" + 0.005*"cheers"'),
 (1,
  '0.007*"moon" + 0.007*"jeff" + 0.006*"ssrt" + 0.006*"crusades" + 0.005*"basically" + 0.004*"really" + 0.004*"pd" + 0.004*"phil" + 0.004*"sun" + 0.004*"nasa"'),
 (2,
  '0.010*"god" + 0.009*"mary" + 0.008*"baptist" + 0.007*"islam" + 0.006*"ns" + 0.006*"zoroastrians" + 0.006*"joseph" + 0.006*"lucky" + 0.005*"khomeini" + 0.005*"samaritan"'),
 (3,
  '0.011*"targa" + 0.010*"whatever" + 0.009*"koresh" + 0.008*"bible" + 0.007*"joy" + 0.007*"thanks" + 0.007*"yayayay" + 0.006*"happy" + 0.006*"craig" + 0.006*"gerald"')]

In [16]:
# check the lsa vector for the first document
corpus_lda = lda_model[corpus_tfidf]
print(corpus_lda[0])

[(0, 0.11853522), (1, 0.7147353), (2, 0.08343477), (3, 0.08329468)]


In [17]:
#predict topics of a new doc
new_doc = "God is love and God is the Lord"
#transform into BOW space
bow_vector = dictionary.doc2bow(preprocess(new_doc))
print([(dictionary[id], count) for id, count in bow_vector])

[('lord', 1), ('god', 2)]


In [18]:
#transform into LDA space
lda_vector = lda_model[bow_vector]
print(lda_vector)

[(0, 0.06270282), (1, 0.06254067), (2, 0.8122362), (3, 0.062520325)]


In [19]:
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

0.010*"god" + 0.009*"mary" + 0.008*"baptist" + 0.007*"islam" + 0.006*"ns" + 0.006*"zoroastrians" + 0.006*"joseph" + 0.006*"lucky" + 0.005*"khomeini" + 0.005*"samaritan"


In [20]:
lda_vector_tfidf = lda_model[tfidf_model[bow_vector]]
print(lda_vector_tfidf)
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector_tfidf, key=lambda item: item[1])[0]))

[(0, 0.10437863), (1, 0.103995994), (2, 0.6876788), (3, 0.103946544)]
0.010*"god" + 0.009*"mary" + 0.008*"baptist" + 0.007*"islam" + 0.006*"ns" + 0.006*"zoroastrians" + 0.006*"joseph" + 0.006*"lucky" + 0.005*"khomeini" + 0.005*"samaritan"


# Latent Semantic Indexing (LSI)

In [21]:
from gensim.models.lsimodel import LsiModel

#It takes a long time

# train the lsi model, choosing number of topics equal to 20


lsi_model = LsiModel(corpus_tfidf, num_topics=4, id2word=dictionary)

In [22]:
# check the topics
lsi_model.print_topics(4)

[(0,
  '-0.769*"god" + -0.346*"jesus" + -0.235*"bible" + -0.204*"christian" + -0.148*"christians" + -0.107*"christ" + -0.090*"well" + -0.085*"koresh" + -0.081*"kent" + -0.080*"christianity"'),
 (1,
  '0.863*"thanks" + 0.255*"please" + 0.160*"hello" + 0.152*"hi" + -0.123*"god" + 0.112*"sorry" + 0.088*"could" + 0.073*"windows" + 0.068*"jpeg" + 0.063*"gif"'),
 (2,
  '0.781*"well" + -0.229*"god" + 0.163*"yes" + -0.154*"thanks" + 0.133*"ico" + 0.133*"tek" + 0.130*"queens" + 0.130*"beauchaine" + 0.130*"bronx" + 0.129*"manhattan"'),
 (3,
  '0.339*"well" + -0.336*"ico" + -0.334*"tek" + -0.328*"beauchaine" + -0.328*"bronx" + -0.328*"queens" + -0.326*"manhattan" + -0.306*"com" + -0.304*"bob" + -0.072*"god"')]

In [23]:
# check the lsi vector for the first document
print(corpus_tfidf[0])

[(0, 0.24093628445650234), (1, 0.5700978153855775), (2, 0.10438175896914427), (3, 0.1598114653031772), (4, 0.722808853369507), (5, 0.24093628445650234)]


# References

* [NLTK Book. Natural Language Processing with Python. Steven Bird, Ewan Klein, and Edward Loper. O'Reilly Media, 2009 ](http://www.nltk.org/book_1ed/)
* [NLTK Essentials, Nitin Hardeniya, Packt Publishing, 2015](http://proquest.safaribooksonline.com/search?q=NLTK%20Essentials)

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© Carlos A. Iglesias, Universidad Politécnica de Madrid.